# Crossref pipeline

We have a simple task, we want to retireve a list of papers that is close to the universe of papers for a particular search string (using the paper title). We are going to use Crossref data and use it to query and get the publication DOIs and download the corresponding PDFs when is possible (we're doing low-key a crime here by using SciHub as a source of PDFs, but more on that later). 

In [44]:
import os
import duckdb
from scidownl import scihub_download

### How many papers in adaptation we have in the Crossref database by year?

In [51]:
duckdb.query(
    """
    SELECT title, 
    type, 
    url, 
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true)
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change')  
    AND type = 'journal-article' order by 4;
    """
)

┌──────────────────────┬───────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │                      column_type                      │  null   │   key   │ default │  extra  │
│       varchar        │                        varchar                        │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼───────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ given_1              │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ given_2              │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ given_3              │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ given_4              │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ family_1             │ VARCHAR

In [52]:
# Try to get all the stuff we need!
adapt_papers = duckdb.query(
    """
    SELECT title, 
    container_title,
    doi, 
    url,
    family_1,
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true)
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change')  
    AND type = 'journal-article' order by 4;
    """
).to_df()

adapt_papers.head()

,title,container_title,doi,url,family_1,year
0,Human adaptation to climate change: An introdu...,American Journal of Human Biology,10.1002/ajhb.23530,http://dx.doi.org/10.1002/ajhb.23530,Pisor,2020
1,Beyond skeletal studies: A computational analy...,American Journal of Biological Anthropology,10.1002/ajpa.24932,http://dx.doi.org/10.1002/ajpa.24932,Bastir,2024
2,The spatial representation of business models ...,Business Strategy &amp; Development,10.1002/bsd2.92,http://dx.doi.org/10.1002/bsd2.92,DiBella,2019
3,Business adaptation to climate change: America...,Business Strategy and the Environment,10.1002/bse.2316,http://dx.doi.org/10.1002/bse.2316,Rivera,2019
4,A relational view of climate adaptation in the...,Business Strategy and the Environment,10.1002/bse.2375,http://dx.doi.org/10.1002/bse.2375,Canevari‐Luzardo,2019


In [54]:
def download_paper(doi_url, title, lastname, path_pdfs):
    file_name = f"{'_'.join([lastname, title.split(' ')[0]])}.pdf"

    out = os.path.join(path_pdfs, file_name)
    proxies = {"http": "socks5://127.0.0.1:7890"}
    scihub_download(doi_url, paper_type="doi", out=out, proxies=proxies)

In [55]:
for idx, row in adapt_papers.iterrows():
    download_paper(row["doi"], row["title"], row["family_1"], "./pdfs/")

[INFO] | 2024/10/01 07:42:30 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:42:30 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/ajhb.23530], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:42:33 | -> Response: status_code=200, content_length=7842
[INFO] | 2024/10/01 07:42:33 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-11-25/da/10.1002@ajhb.23530.pdf', 'title': 'Human adaptation to climate change  An introduction to the special issue. American Journal of Human Biology, 33(4)'}


100% [==================================================] 1327306/1327306


[INFO] | 2024/10/01 07:42:34 | ↓ Successfully download the url to: ./pdfs/Pisor_Human.pdf
[INFO] | 2024/10/01 07:42:34 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:42:34 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/ajpa.24932], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:42:35 | -> Response: status_code=200, content_length=5827
[WARNING] | 2024/10/01 07:42:35 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/10/01 07:42:35 | Choose scihub url [1]: http://sci-hub.se
[INFO] | 2024/10/01 07:42:35 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1002/ajpa.24932], proxies={'http': 'socks5://127.0.0.1:7890'}
[WARNING] | 2024/10/01 07:42:35 | Error occurs, task status: crawling_failed, error: SOCKSHTTPConnectionPool(host='sci-hub.se', port=80): Max retries exceeded with url: / (Caused by NewConnectio

100% [==================================================] 18025086/18025086


[INFO] | 2024/10/01 07:42:41 | ↓ Successfully download the url to: ./pdfs/DiBella_The.pdf
[INFO] | 2024/10/01 07:42:41 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:42:41 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/bse.2316], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:42:42 | -> Response: status_code=200, content_length=7833
[INFO] | 2024/10/01 07:42:42 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2019-10-23/05/rivera2019.pdf', 'title': 'Business adaptation to climate change  American ski resorts and warmer temperatures. Business Strategy and the Environment'}


...

[INFO] | 2024/10/01 07:44:17 | ↓ Successfully download the url to: ./pdfs/Kwakkel_Is.pdf
[INFO] | 2024/10/01 07:44:17 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:44:17 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/wcc.676], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:44:18 | -> Response: status_code=200, content_length=7829
[INFO] | 2024/10/01 07:44:18 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-08-18/72/lucas2020.pdf', 'title': 'Privatizing climate adaptation  How insurance weakens solidaristic and collective disaster recovery. WIREs Climate Change'}


100% [==================================================] 2134073/2134073


[INFO] | 2024/10/01 07:44:19 | ↓ Successfully download the url to: ./pdfs/Lucas_Privatizing.pdf
[INFO] | 2024/10/01 07:44:20 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:44:20 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/wcc.695], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:44:20 | -> Response: status_code=200, content_length=7791
[INFO] | 2024/10/01 07:44:20 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-12-17/83/tubi2020.pdf', 'title': 'Beyond binary outcomes in climate adaptation  The illustrative case of desalination. WIREs Climate Change'}


100% [==================================================] 2708058/2708058


[INFO] | 2024/10/01 07:44:22 | ↓ Successfully download the url to: ./pdfs/Tubi_Beyond.pdf
[INFO] | 2024/10/01 07:44:22 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:44:22 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.1002/wcc.856], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:44:23 | -> Response: status_code=200, content_length=5824
[WARNING] | 2024/10/01 07:44:23 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/10/01 07:44:23 | Choose scihub url [1]: http://sci-hub.se
[INFO] | 2024/10/01 07:44:23 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1002/wcc.856], proxies={'http': 'socks5://127.0.0.1:7890'}
[WARNING] | 2024/10/01 07:44:23 | Error occurs, task status: crawling_failed, error: SOCKSHTTPConnectionPool(host='sci-hub.se', port=80): Max retries exceeded with url: / (Caused by NewConnectionError

In [9]:
duckdb.query(
    """
    SELECT 
    title, 
    type, 
    url, 
    abstract, 
    list_extract(list_extract(published['date-parts'], 1),1) as year 
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true) 
    WHERE regexp_matches(title, 'climate adaptation|adaptation to climate change') AND 
    type = 'journal-article' and abstract IS NOT NULL
    ORDER by 5;
    """
)

┌──────────────────────┬─────────────────┬──────────────────────┬──────────────────────────────────────────────┬───────┐
│        title         │      type       │         url          │                   abstract                   │ year  │
│       varchar        │     varchar     │       varchar        │                   varchar                    │ int64 │
├──────────────────────┼─────────────────┼──────────────────────┼──────────────────────────────────────────────┼───────┤
│ Reducing hazard vu…  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>O…  │  2006 │
│ Community‐based di…  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>C…  │  2006 │
│ Local initiatives …  │ journal-article │ http://dx.doi.org/…  │ <jats:title>Abstract</jats:title><jats:p>C…  │  2006 │
│ Modeling agricultu…  │ journal-article │ http://dx.doi.org/…  │ <jats:p>An approach that integrates biophy…  │  2008 │
│ A multi-institutio…  │ journal

In [12]:
duckdb.query(
    """
    with clean as (
        SELECT list_extract(affiliation_1, 1) as first_author_affiliation,
        list_extract(list_extract(published['date-parts'], 1),1) as year
        FROM read_parquet('/mnt/drive/crossref_april_parquet_normalized/*.parquet', union_by_name=true)
        WHERE affiliation_1 IS NOT NULL
        ) SELECT first_author_affiliation, count(*) AS count 
        FROM clean 
        WHERE first_author_affiliation IS NOT NULL and year == 2020
        GROUP BY first_author_affiliation 
        ORDER by count desc;
    """
)

┌─────────────────────────────────────────────────────────────────────────────────────────┬───────┐
│                                first_author_affiliation                                 │ count │
│                                         varchar                                         │ int64 │
├─────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ University of Washington                                                                │ 37393 │
│ Department of Chemistry                                                                 │  3678 │
│ California Institute of Technology                                                      │  1110 │
│ University of Massachusets Medical School                                               │  1075 │
│ Los Alamos National Lab. (LANL), Los Alamos, NM (United States)                         │  1003 │
│ Stanford University                                                                     │   828 │


In [25]:
duckdb.query(
    """
    SELECT container_title, title, type, url, list_extract(list_extract(published['date-parts'], 1),1) as year
    FROM read_parquet('/mnt/drive/crossref_april_parquet_normalized/*.parquet', union_by_name=true)
    WHERE contains(container_title, 'development')
    and list_extract(list_extract(published['date-parts'], 1),1) == 2020
    and type = 'journal-article'
    """
)

┌──────────────────────┬──────────────────────┬─────────────────┬──────────────────────────────────────────────┬───────┐
│   container_title    │        title         │      type       │                     url                      │ year  │
│       varchar        │       varchar        │     varchar     │                   varchar                    │ int64 │
├──────────────────────┼──────────────────────┼─────────────────┼──────────────────────────────────────────────┼───────┤
│ Advances in Neurod…  │ Building Social Sk…  │ journal-article │ http://dx.doi.org/10.1007/s41252-020-00149-8 │  2020 │
│ Journal of Neurode…  │ A telehealth appro…  │ journal-article │ http://dx.doi.org/10.1186/s11689-019-9302-0  │  2020 │
│ Journal of Neurode…  │ Health comorbiditi…  │ journal-article │ http://dx.doi.org/10.1186/s11689-019-9306-9  │  2020 │
│ Advances in Neurod…  │ Mindfulness Traini…  │ journal-article │ http://dx.doi.org/10.1007/s41252-020-00148-9 │  2020 │
│ South of Russia: e…  │ Ecologi

In [34]:
from scidownl import scihub_download

paper = "https://doi.org/10.1145/3375633"
paper_type = "doi"
out = "./paper/one_paper.pdf"
proxies = {"http": "socks5://127.0.0.1:7890"}
scihub_download(paper, paper_type=paper_type, out=out, proxies=proxies)

[INFO] | 2024/10/01 07:08:00 | Found 6 valid SciHub domains in total: ['https://sci-hub.ru', 'http://sci-hub.se', 'https://sci-hub.se', 'http://sci-hub.ru', 'http://sci-hub.st', 'https://sci-hub.st']
[INFO] | 2024/10/01 07:08:00 | Saved 6 SciHub domains to local db.
[INFO] | 2024/10/01 07:08:00 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/10/01 07:08:00 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=doi.org/10.1145/3375633], proxies={'http': 'socks5://127.0.0.1:7890'}
[INFO] | 2024/10/01 07:08:02 | -> Response: status_code=200, content_length=7868
[INFO] | 2024/10/01 07:08:02 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2021-06-09/4a/beschastnikh2020.pdf', 'title': 'Visualizing Distributed System Executions. ACM Transactions on Software Engineering and Methodology, 29(2), 1–38'}


100% [==================================================] 2522115/2522115


[INFO] | 2024/10/01 07:08:03 | ↓ Successfully download the url to: ./paper/one_paper.pdf


In [26]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("http://dx.doi.org/10.1007/s41252-020-00151-0")

In [33]:
import pprint

docs = loader.load()

pprint.pp(docs[0].page_content)

('\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Factors Influencing Decisions About Prenatal Genetic Testing for Autism '
 'Among Mothers of Children with Autism Spectrum Disorders | Advances in '
 'Neurodevelopmental Disorders\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'

In [ ]:
duckdb.query(
    """
    SELECT unnest(CAST(affiliation_1 AS VARCHAR[]), recursive := true)
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true)
    WHERE affiliation_1 IS NOT NULL
    LIMIT 10;
    """
)

In [5]:
duckdb.query(
    """
    with clean_year as (
    select *, list_extract(list_extract(published['date-parts'], 1),1) as year,
    FROM read_parquet('/mnt/drive/crossref_april_parquet/*.parquet', union_by_name=true) 
    ) select year, affiliation_1, count(*)
    from clean_year
    where affiliation_1 is not null
    group by year, affiliation_1;
    """
)

TypeMismatchException: Mismatch Type Error: Type STRUCT("name" VARCHAR, place VARCHAR[]) does not match with STRUCT("name" VARCHAR). Cannot cast STRUCTs of different size